# The SignalSystem and recorded Sessions in depth

* System thread and update_rate
* Adding functions to live system
* data feeds
* recorders, file types
* metadata, sessions
* creating a PyTorch dataset / pandas dataframe etc. from sessions (with computed signal functions)?